In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as ply
import sys, os

<span style="color: Gold"> 1. 패치분할

In [ ]:
def patch_embedding():
    '''이미지를 패치로 분할하는 과정(patch embedding)'''
    # 설정
    image_size = 224
    patch_size = 16
    channels = 3
    embedding_dim = 768

    # 패치수 계산
    num_patchs = (image_size // patch_size) **2
    print(f' 이미지 크기 : {image_size} x {image_size }')
    print(f' 패치 크기 : {patch_size} x {patch_size }')
    print(f' 채널 수 : {channels}')
    print(f' 패치 수 : {image_size // patch_size} x{image_size // patch_size}')

    # 더미 이미지 생성
    dummy_image = torch.randn(1,channels, image_size, image_size)
    print(f' 더미 이미지 생성')
    print(f' 입력 이미지 shape : {dummy_image.shape}') # [1,3,224,224]

    # 패치분할(conv2 사용)
    # conv2 stride = patch_size 겹치지 않는 패치 추출 / # stride = 커널이 한 번에 몇 칸씩 이동하느냐
    patch_embed = nn.Conv2d(in_channels=channels, out_channels=embedding_dim, kernel_size=patch_size, stride=patch_size) # 입력하는 채널의 수, 출력하는 채널의 수, kernel_size -> 몇개씩 도장을 찍느냐, stride -> 이동 간격

    # 패치 임베딩 적용
    patches = patch_embed(dummy_image)
    print(f' \n 패치 임베딩 후')
    print(f' conv2d 출력 shape : {patches.shape}') # [1, 768, 14, 14]

    # Flatten : (배치사이즈(B), 임베딩 차이(D), 이미지 높이(H), 이미지 넓이(W)) -> (B, 196(HxW), D) -> (1, 196, 768)
    patches_flat = patches.flatten(2).transpose(1,2)
    print(f' Flatten 후 shape : {patches_flat.shape}') # [1, 196, 768]

    # 각 패치는 768차원 벡터
    print(f' \n 패치 수  : {patches_flat.shape[1]}')
    print(f' 각 패치의 임베딩 차원 수  : {patches_flat.shape[2]}')
    return patches_flat

if __name__ == '__main__':
    patch_embedding()

 이미지 크기 : 224 x 224
 패치 크기 : 16 x 16
 채널 수 : 3
 패치 수 : 14 x14
 더미 이미지 생성
 입력 이미지 shape : torch.Size([1, 3, 224, 224])
 
 패치 임베딩 후
 conv2d 출력 shape : torch.Size([1, 768, 14, 14])
 Flatten 후 shape : torch.Size([1, 196, 768])
 
 패치 수  : 196
 각 패치의 임베딩 차원 수  : 768


In [ ]:
# 위치 임베딩의 역할
def positional_embedding():
    '''위치 임베딩'''
    num_patches = 196
    embedding_dim = 768

    # 위치 임베딩 생성  
    # 이 텐서는 학습대상 Optimizer 에 의해 업데이트
    position_embedding = nn.Parameter(torch.randn(1, num_patches+1, embedding_dim)) # num_patches+1 -> +1을 하는 이유는 cls 토큰을 추가하기 위해서
    print(f' 위치 임베딩 shape : {position_embedding.shape}')
    print(f' 총 위치 수 : {num_patches+1} (패치 196 + cls 토큰 1개)')

    # 배치 차원 제거 -> 각 위치를 하나의 벡터로 다루기 위해 배치 크기가 1인 형태는 분석 시 불필요
    pos_emb = position_embedding.squeeze(0) #squeeze(0)-> 첫번째(배치크기)를 숨긴다